<a href="https://colab.research.google.com/github/Santigs/DS-Course/blob/main/Lecture5e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gql

In [2]:
import pandas as pd
import plotly.express as px

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


In [ ]:
#---------- Extract data from Uniswap Subgraph ----------

sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
    verify=True,
    retries=3,
)
client = Client(
    transport=sample_transport
)
query = gql('''
query {
    pairs (first: 5) {
    token0 {
      symbol
    }
    token1 {
      symbol
    }
  }
}
''')
response1 = client.execute(query)
print(response1)
print(response1['pairs'])

In [ ]:
# ----------- Creating the dataframe from the json file. -------------

pairs = []
for i in response1['pairs']:
    pairs.append([
        i['token0']['symbol'],
        i['token1']['symbol'],
     ])

print(pairs)

df = pd.DataFrame(pairs)
df

In [ ]:
# ----------- Creating a different query -------------

query = gql('''
query {
      pairs (first: 50, where: {volumeUSD_gt: "10000000"}) 
    {
    volumeUSD
    token0 {
      symbol
    }
    token1 {
      symbol
    }
  }
}
''')
response2 = client.execute(query)

print()
print()

pairs = []
for i in response2['pairs']:
    pairs.append([
        i['token0']['symbol'],
        i['token1']['symbol'],
        i['volumeUSD']
     ])


df = pd.DataFrame(pairs)
df.head()


In [ ]:
df[2] = df[2].astype(float)
df.columns=['Token1','Token2', 'Volume-USD']
df.head()

In [ ]:
df['Pair']=df['Token1']+'-'+df['Token2']
df.head()

In [ ]:
fig = px.bar(df, x='Pair', y='Volume-USD')
fig.show()